In [1]:
import os
import dlib
import cv2 as cv
import face_recognition as fr
import numpy as np
from PIL import Image, ImageDraw, ImageFont

## 训练

In [2]:
# 训练集结果存储
Face_encodings = []
Face_names = []

root_dir = "./example"

# 读取所有训练数据进行训练
for file in os.listdir(root_dir):
    file_name = root_dir + "/" + file    
    image = fr.load_image_file(file_name)
    face_encoding = fr.face_encodings(image)[0]
    Face_encodings.append(face_encoding)
    (Name, extension) = os.path.splitext(file)
    Face_names.append(Name)

In [3]:
# 在图片上输出中文字符
def cv2ImgAddText(img, text, left, top, textColor=(255, 255, 255), textSize=20):
    if (isinstance(img, np.ndarray)):  # 判断是否OpenCV图片类型
        img = Image.fromarray(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    # 创建一个可以在给定图像上绘图的对象
    draw = ImageDraw.Draw(img)
    # 字体的格式
    fontStyle = ImageFont.truetype("font/simsun.ttc", textSize, encoding="utf-8")
    # 绘制文本
    draw.text((left, top), text, textColor, font=fontStyle)
    # 转换回OpenCV格式
    return cv.cvtColor(np.asarray(img), cv.COLOR_RGB2BGR)


In [4]:
# 调用笔记本摄像头，参数一般为0
# cap是摄像头对象
cap = cv.VideoCapture(0)

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True


# 在while循环中，利用摄像头对象的read()函数读取视频的某帧并显示。
# 然后等待1个单位时间，如果期间检测到了键盘输入q，则退出，即关闭窗口。
while(1):
    try:
        ret, frame = cap.read()
        if True == ret:   
            # 压缩图片到1/4大小，加快识别速度
            small_frame = cv.resize(frame, (0, 0), fx=0.25, fy=0.25)

            # OpenCV  BGR
            # face_cognition RGB
            # BGR  to    RGB
            rgb_small_frame = small_frame[:, :, ::-1]

            if process_this_frame:
                # 找出所有的人脸 与 人脸特征向量
                face_locations = fr.face_locations(rgb_small_frame)
                face_encodings = fr.face_encodings(rgb_small_frame, face_locations)

                face_names = []
                for face_encoding in face_encodings:
                    # 与已知的人脸特征向量比对
                    matches = fr.compare_faces(Face_encodings, face_encoding)
                    name = u"Unknown"

                    face_distances = fr.face_distance(Face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)
                    if matches[best_match_index]:
                        name = Face_names[best_match_index]
                    face_names.append(name)

            process_this_frame = not process_this_frame


            # 展示结果
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                # 转回原来的大小
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                # 矩形框脸
                cv.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

                # 脸下方画标签
                cv.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv.FILLED)
                font = cv.FONT_HERSHEY_DUPLEX
                
                # opencv 不支持汉字标签
                #cv.putText(frame,name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
                frame =  cv2ImgAddText(frame,name,(left+(right-left)/4), bottom - 30, (255, 0, 0),30)

            cv.imshow('FaceRecognition', frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
    except Exception as e:
        print(e)
        break
            
            
# 释放摄像头对象，关闭所有图像窗口
cap.release()
cv.destroyAllWindows()